In [2]:
import pandas as pd
data = pd.read_json("output.json.gz", orient="table")
from rdkit import Chem

FileNotFoundError: File output.json.gz does not exist

In [3]:
import pandas as pd
from aizynthfinder.reactiontree import ReactionTree

data = pd.read_json("output.json", orient="table")
all_trees = data.trees.values  # This contains a list of all the trees for all the compounds
print(len(all_trees))
trees_for_first_target = all_trees[0]
print(trees_for_first_target)

# Create a list to store dictionaries containing reaction information
all_rxn = []
count = 0
for molecule in all_trees:
    count += 1
    for itree, tree in enumerate(molecule):
        print("this is new route")
        imagefile = f"molecule{count}route{itree:03d}.png"
        reaction_tree = ReactionTree.from_dict(tree)
        ReactionTree.from_dict(tree).to_image().save(imagefile)

        # Print the reaction SMILES
        rxn_generator = reaction_tree.reactions()
        for rxn in rxn_generator:
            try:
                smiles_reaction = rxn.to_smiles_based_retroreaction()
                template = rxn.metadata["template"]
                template_hash = rxn.metadata["template_hash"]
                
                # Store all information in a dictionary
                reaction_info = {
                    'smiles_reaction': smiles_reaction,
                    'template': template,
                    'template_hash': template_hash
                }
                all_rxn.append(reaction_info)
                
                # Print information for debugging
                print(f"This is rxn: {smiles_reaction}")
                print(f"this is template hash: {template_hash}")
                print(f"this is template: {template}")
            except Exception as e:
                print(f"Error: {e}")
                print(f"Error in {rxn.metadata}")

1
[{'type': 'mol', 'hide': False, 'smiles': 'O=C1CCCCC2=C1CCC2', 'is_chemical': True, 'in_stock': False, 'children': [{'type': 'reaction', 'hide': False, 'smiles': '[C:1]([CH3:2])[CH:3]=[O:4]>>CCOC(=O)[C:1]([CH3:2])[CH:3]=[O:4]', 'is_reaction': True, 'metadata': {'template_hash': '8e15c52784a3d916418c2e16d27813299b9b75ade775e9baa5b88ca1ac39b50d', 'classification': '0.0 Unrecognized', 'library_occurence': 72, 'policy_probability': 0.1088000014, 'policy_probability_rank': 1, 'policy_name': 'uspto', 'template_code': 23380, 'template': '[C:2]-[CH2;D2;+0:1]-[C:3]=[O;D1;H0:4]>>C-C-O-C(=O)-[CH;D3;+0:1](-[C:2])-[C:3]=[O;D1;H0:4]', 'mapped_reaction_smiles': '[O:1]=[C:2]1[CH2:3][CH2:4][CH2:5][CH2:6][C:7]2=[C:8]1[CH2:9][CH2:10][CH2:11]2>>[O:1]=[C:2]1[CH:3]([C:12]([O:13][CH2:14][CH3:15])=[O:16])[CH2:4][CH2:5][CH2:6][C:7]2=[C:8]1[CH2:9][CH2:10][CH2:11]2'}, 'children': [{'type': 'mol', 'hide': False, 'smiles': 'CCOC(=O)C1CCCC2=C(CCC2)C1=O', 'is_chemical': True, 'in_stock': False, 'children': [{'type

In [6]:
# Create DataFrame with the collected information
df = pd.DataFrame(all_rxn)

# Save to CSV with specified columns
df.to_csv('rxn.csv', index=False)

print(f"Exported {len(df)} reactions to rxn.csv")

Exported 305 reactions to rxn.csv
